In [1]:
# !pip install beautifulsoup4 selenium webdriver-manager

# import requests
from bs4 import BeautifulSoup

# headless browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# read zipfile in memory
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# parsing 
import pandas as pd
import p_tqdm as tqdm


### In this section we use the chrome webbrowser to get a list of all available links.

- We need to do this rather than use a request, as the page structure is populated by javascript and not the html source code itself. 
- This means we open the browser, load the code, get the source and close it again. 

In [2]:
URL = "https://www.nomisweb.co.uk/census/2011/bulk/r2_2#KeyStatistics"
domain = 'https://www.nomisweb.co.uk/'
loc = '../DATA/'
prefix = 'OA11/'
os.system('mkdir '+loc+prefix)

#  lets create a fake headless browser


chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get(URL)

soup = BeautifulSoup(driver.page_source, "html.parser")

driver.close()


mkdir: 

../DATA/OA11/: File exists
====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/danielellis/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache
/var/folders/h8/t803dxss41g9kj07lkgx95fm0000gn/T/ipykernel_71523/3262914912.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [3]:
# soup.find("table", {"class": "table table-striped table-bordered table-condensed"})
table = soup.find('tbody')
table = list(filter(lambda x: x!= '\n',table))

Now we have the source code, we filter out the links. 

In [4]:
level = 0 
filelist = []


for i in table: 
    # // filter to key statistics only
    if len(i)<2: 
        print(i.find('a')) ; level += 1
        if level >1 : break
        continue
    
    select = i.findAll('td')
    if '-' in select[1].text:
        continue # skip male female breakdown. 
    if 'Welsh' in select[1].text:
        continue # skip welsh language skills (too specific). 

    obj = dict(code = select[0].text, name = select[1].text, url = select[3].find('a').get('href'))
    filelist.append(obj)
    

<a id="KeyStatistics" name="KeyStatistics"></a>
<a id="QuickStatistics" name="QuickStatistics"></a>


In [5]:
# lets check the last fiew values
filelist[-3:] 

[{'code': 'KS604EW',
  'name': 'Hours Worked',
  'url': '/output/census/2011/ks604ew_2011_oa.zip'},
 {'code': 'KS605EW',
  'name': 'Industry',
  'url': '/output/census/2011/ks605ew_2011_oa.zip'},
 {'code': 'KS608EW',
  'name': 'Occupation',
  'url': '/output/census/2011/ks608ew_2011_oa.zip'}]

### The next step lies extracting the data:
1. Download zip into memory 
2. Extract whist still in RAM
3. Concatenate tables in Pandas
4. Use description table to give this useful names. 
5. Save to disk. 

In [14]:

# indicator = filelist[3]
def extract (indicator):
    ''' A function to extract the data into memory, process it and, and then save into csvs. '''
    try: 
        resp = urlopen(domain+indicator['url'])
        zipfile = ZipFile(BytesIO(resp.read()))
    except: # occasionally an in-memory download may fail. Trigger a restart
        print('\n\n---',indicator)
        try:
            resp = urlopen(domain+indicator['url'])
            zipfile = ZipFile(BytesIO(resp.read()))
        except:
            print('\n\n\nFailed',indicator)
            return

    # lets read all the data into a single df
    namelist = filter(lambda x: 'DATA' in x , zipfile.namelist())
    all_d = pd.concat([pd.read_csv(zipfile.open(f)) for f in namelist], axis=0, ignore_index=True).set_index('GeographyCode')


    #  Exctract Headers from the DESCRIPTION file
    headers = pd.read_csv(zipfile.open(filter(lambda x: 'DESC' in x , zipfile.namelist()).__next__())).set_index('ColumnVariableCode')
    headers  = headers[headers.ColumnVariableMeasurementUnit=='Count']['ColumnVariableDescription'].to_dict() #  filter to counts only

    # only take columns we have count values for
    all_d = all_d[[k for k in all_d.columns if (k in headers)]] 
    # rename columns
    all_d.columns = [headers[k] for k in all_d.columns]
    # we do not want summed counts as they ruin percentages
    for c in filter(lambda x: x[:14]=='All categories',all_d.columns): all_d.drop(c,inplace=True,axis=1)

    all_d.to_csv(loc+prefix+indicator['code']+'_'+indicator['name'].replace(" ","_").lower()+'.csv')

    # print(indicator)



As usual we can run this in parallel as it is is an 'embarassingly parallel' problem

In [15]:
_ = tqdm.p_map(extract,filelist)

100%|██████████| 24/24 [03:18<00:00,  8.29s/it]




--- {'code': 'KS102EW', 'name': 'Age Structure', 'url': '/output/census/2011/ks102ew_2011_oa.zip'}


--- {'code': 'KS201EW', 'name': 'Ethnic Group', 'url': '/output/census/2011/ks201ew_2011_oa.zip'}
